### Parametry ogólne

In [2]:
import random
from sympy import isprime
from collections import defaultdict
from abc import ABC, abstractmethod

In [4]:
# ==========================================================
# LISTY I FUNKCJE POMOCZNICZE
# ==========================================================

binaries = ""
for i in range(1000):
    binaries += bin(i)[2:]
binaries = [int(binaries[i]) for i in range(len(binaries))]

perfect_numbers = [6, 28, 496, 8128, 33550336]

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))

# ==========================================================
# KLASA BAZOWA STRATEGII
# ==========================================================

class Strategy(ABC):
    def __init__(self, name):
        self.name = name

    def reset(self):
        """Reset stanu przed nową grą"""
        pass

    @abstractmethod
    def ruch(self, my_history, opp_history):
        """Zwraca 'L' albo 'Z'"""
        pass

# ==========================================================
# PRZYKŁADOWE STRATEGIE
# ==========================================================

class ZawszeLojalnosc(Strategy):
    def ruch(self, my_history, opp_history):
        return 'L'


class ZawszeZdrada(Strategy):
    def ruch(self, my_history, opp_history):
        return 'Z'


class WetZaWet(Strategy):
    def ruch(self, my_history, opp_history):
        if not opp_history:
            return 'L'
        return opp_history[-1]


class FochForever(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.foch = False

    def reset(self):
        self.foch = False

    def ruch(self, my_history, opp_history):
        if 'Z' in opp_history:
            self.foch = True
        return 'Z' if self.foch else 'L'


class FiftyFifty(Strategy):
    def ruch(self, my_history, opp_history):
        return random.choice(['L', 'Z'])


class TrynidadTobago(Strategy):
    def ruch(self, my_history, opp_history):
        if random.random() < 0.1:
            return 'Z'
        else:
            if not opp_history:
                return 'L'
            return opp_history[-1]


class MetodaKubanska(Strategy):
    def ruch(self, my_history, opp_history):
        if len(my_history) < 10:
            return random.choice(['L', 'Z'])
        else:
            p = opp_history.count('Z')/len(opp_history)
            return random.choices(['L', 'Z'], weights=[1-p, p])[0]


class PrawieSiecNeuronowa(Strategy):
    def ruch(self, my_history, opp_history):
        n = len(my_history)
        return 'L' if binaries[n] == 0 else 'Z'
        

class DarwoX(Strategy):
    def ruch(self, my_history, opp_history):
        return 'Z' if isprime(len(my_history)+1) else 'L'


class PrzebaczajacyDoCzasu(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.foch = False

    def reset(self):
        self.foch = False

    def ruch(self, my_history, opp_history):
        if len(opp_history) > 2:
            if opp_history[-3:] == ['Z','Z','Z']:
                self.foch = True
        if self.foch:
            return 'Z'
        if not opp_history:
            return 'L'
        return opp_history[-1]


class Reklama(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.streak = 2
        self.numstreak = 0

    def reset(self):
        self.streak = 2
        self.numstreak = 0

    def ruch(self, my_history, opp_history):
        if self.numstreak == 0:
            if len(opp_history) > 1:
                if opp_history[-self.streak:] == ['Z']*self.streak:
                    self.numstreak += 1
                    return 'Z'
                else:
                    return opp_history[-1]   
            if not opp_history:
                return 'L'
            return opp_history[-1]
        else:
            self.numstreak += 1
            if self.numstreak == self.streak:
                self.numstreak == 0
                self.streak += 1
            return 'Z'


class NieLubiPrzegrywać(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.my_score = 0
        self.opp_score = 0

    def reset(self):
        self.my_score = 0
        self.opp_score = 0
        
    def ruch(self, my_history, opp_history):
        if len(my_history) > 0:
            self.my_score += MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][0]
            self.opp_score += MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][1]
        if self.my_score > self.opp_score:
            return 'L'
        return 'Z'


class KubaSubcio(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.my_score = 0

    def reset(self):
        self.my_score = 0
        
    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'Z'
        self.my_score += MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][0]
        if self.my_score/len(my_history) > -5 and self.my_score/len(my_history) < -3:
            return 'L'
        return 'Z'


class PNANG(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.runda = 0

    def reset(self):
        self.runda = 0

    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'L'
        if 'Z' in opp_history:
            if not self.runda:
                self.runda = len(my_history)
            p = 2.71828**(-0.1*(len(my_history)-self.runda))
            return random.choices(['L','Z'],weights=[1-p,p])[0]
        return 'L'   


class OdsiewFrajerow(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.foch = False
        self.lojalne = 0

    def reset(self):
        self.foch = False
        self.lojalne = 0
        
    def ruch(self, my_history, opp_history):
        if len(my_history) != 0 and not self.foch:
            if 'Z' in opp_history:
                self.foch == True
        if not self.foch:
            return 'Z'
        if self.lojalne < 3:
            self.lojalne += 1
            return 'L'
        return opp_history[-1]


class Jedikarix(Strategy):
    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'Z'
        if opp_history.count('L') > opp_history.count('Z'):
            return 'L'
        if opp_history.count('L') < opp_history.count('Z'):
            return 'Z'
        return random.choice(['L', 'Z'])


class Krukindzek(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.lojalny_score = 0
        self.zdrada_score = 0

    def reset(self):
        self.lojalny_score = 0
        self.zdrada_score = 0
        
    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'Z'
        if my_history[-1] == 'L':
            self.lojalny_score += MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][0]
        else:
            self.zdrada_score += MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][0]
        if self.lojalny_score/len(my_history) > self.zdrada_score/len(my_history):
            return 'L'
        if self.lojalny_score/len(my_history) < self.zdrada_score/len(my_history):
            return 'Z'
        return random.choice(['L', 'Z'])


class OrzechowaSzansa(Strategy):
    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'L'
        if len(my_history) > 0 and len(my_history) < 5:
            return opp_history[-1]
        if opp_history[-2:] == ['Z','Z']:
            return 'Z'
        p = opp_history.count('Z')/len(opp_history)
        if p <= 0.25:
            return 'L'
        if p > 0.5:
            return 'Z'
        return opp_history[-1]


class PruskiKwadrat(Strategy):
    def ruch(self, my_history, opp_history):
        if len(my_history)+1 in perfect_numbers:
            return 'Z'
        return 'L'


class Dwulicowiec(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.lojalny_streak = 0

    def reset(self):
        self.lojalny_streak = 0
        
    def ruch(self, my_history, opp_history):
        if not my_history:
            self.lojalny_streak += 1
            return 'L'
        if opp_history[-1] == 'Z' or self.lojalny_streak == 3:
            self.lojalny_streak = 0
            return 'Z'
        self.lojalny_streak += 1
        return 'L'


class ShiZ(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.zdrada_streak = 0

    def reset(self):
        self.zdrada_streak = 0
        
    def ruch(self, my_history, opp_history):
        if len(opp_history) < 3:
            return 'L'
        if opp_history[-3:] == ['L']*3 or opp_history[-3:] == ['Z']*3 or self.zdrada_streak == 1 or self.zdrada_streak == 2:
            self.zdrada_streak += 1
            return 'Z'
        self.zdrada_streak = 0
        return 'L'


class ILike3s(Strategy):
    def __init__(self, name):
        super().__init__(name)
        self.my_score = 0

    def reset(self):
        self.my_score = 0
        
    def ruch(self, my_history, opp_history):
        if not my_history:
            return 'L'
        self.my_score += abs(MACIERZ_WYPLAT[(my_history[-1],opp_history[-1])][0])
        if ternary(self.my_score)[0] == '1':
            return 'L'
        return 'Z'   


class WiaraWLudzkosc(Strategy):
    def ruch(self, my_history, opp_history):
        if len(opp_history) < 4:
            return 'L'
        if opp_history[-4:].count('L') > 2:
            return 'L'
        if opp_history[-4:].count('Z') > 2:
            return 'Z'
        p = opp_history.count('Z')/len(opp_history)
        return random.choices(['L','Z'], weights=[1-p,p])[0]

### Turniej Axleroda

In [13]:
# ==========================================================
# PARAMETRY GLOBALNE
# ==========================================================

MACIERZ_WYPLAT = {
    ('L', 'L'): (-3, -3),
    ('L', 'Z'): (-10, 0),
    ('Z', 'L'): (0, -10),
    ('Z', 'Z'): (-5, -5)
}

DELTA = 0.95         # prawdopodobieństwo kontynuacji rundy
N_TURNIEJE = 10_000  # liczba powtórzeń turnieju
    
# ==========================================================
# POJEDYNCZA GRA ITEROWANA
# ==========================================================

def play_iterated_pd(player1, player2, payoff_matrix, delta):
    h1, h2 = [], []
    score1, score2 = 0, 0

    player1.reset()
    player2.reset()

    while True:
        a1 = player1.ruch(h1, h2)
        a2 = player2.ruch(h2, h1)

        p1, p2 = payoff_matrix[(a1, a2)]
        score1 += p1
        score2 += p2

        h1.append(a1)
        h2.append(a2)

        if random.random() > delta:
            break

    return score1, score2

# ==========================================================
# JEDEN TURNIEJ (ROUND-ROBIN)
# ==========================================================

def run_single_tournament(strategies, payoff_matrix, delta):
    scores = {s.name: 0 for s in strategies}

    for i in range(len(strategies)):
        for j in range(i + 1, len(strategies)):
            s1, s2 = strategies[i], strategies[j]
            p1, p2 = play_iterated_pd(s1, s2, payoff_matrix, delta)
            scores[s1.name] += p1
            scores[s2.name] += p2

    return scores

# ==========================================================
# WIELOKROTNE TURNIEJE + PUNKTACJA MIEJSC
# ==========================================================

def run_meta_tournament(strategies, n_tournaments):
    ranking_points = defaultdict(int)
    n = len(strategies)

    for _ in range(n_tournaments):
        scores = run_single_tournament(strategies, MACIERZ_WYPLAT, DELTA)

        ordered = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        for place, (name, _) in enumerate(ordered):
            ranking_points[name] += n - place
    
    return ranking_points

# ==========================================================
# WYŚWIETLANIE WYNIKÓW
# ==========================================================

def print_results(results):
    ordered = sorted(results.items(), key=lambda x: x[1], reverse=True)

    print("\n=== WYNIKI KOŃCOWE TURNIEJU ===\n")
    print(f"{'Miejsce':<12}{'Strategia':<50}{'Punkty'}")
    print("-" * 74)

    for i, (name, pts) in enumerate(ordered, start=1):
        print(f"{i:<12}{name:<50}{pts}")

# ==========================================================
# WYWOŁANIE FUNKCJI
# ==========================================================

if __name__ == "__main__":
    strategie = [
        ZawszeLojalnosc("Zawsze Lojalność"),
        ZawszeZdrada("Zawsze Zdrada"),
        WetZaWet("Wet Za Wet"),
        FochForever("Foch Forever"),
        FiftyFifty("Fifty-Fifty"),
        TrynidadTobago("Trynidad_Tobago"),
        MetodaKubanska("Metoda Kubańska"),
        PrawieSiecNeuronowa("Tutaj Miała Być Sieć Neuronowa"),
        DarwoX("DarwoX (liczby pierwsze)"),
        PrzebaczajacyDoCzasu("Przebaczający Do Czasu"),
        Reklama("Sprawdź kanał autora (Paradoxalnie)"),
        NieLubiPrzegrywać("Nie Lubi Przegrywać"),
        KubaSubcio("Kuba Subcio"),
        PNANG("Pozytywnie Nastawieni Ale Nieufni Gracze"),
        OdsiewFrajerow("Odsiew Frajerów"),
        Jedikarix("Jedikarix"),
        Krukindzek("Krukindzek"),
        OrzechowaSzansa("Orzechowa Szansa"),
        PruskiKwadrat("Pruski Kwadrat"),
        Dwulicowiec("Dwulicowiec"),
        ShiZ("ShiZ"),
        ILike3s("I Like 3's"),
        WiaraWLudzkosc("Wiara w Ludzkość")
    ]

    results = run_meta_tournament(strategie, N_TURNIEJE)
    print_results(results)


=== WYNIKI KOŃCOWE TURNIEJU ===

Miejsce     Strategia                                         Punkty
--------------------------------------------------------------------------
1           Zawsze Zdrada                                     208765
2           Odsiew Frajerów                                   208030
3           Kuba Subcio                                       195033
4           Krukindzek                                        159887
5           Nie Lubi Przegrywać                               158171
6           Jedikarix                                         156786
7           Tutaj Miała Być Sieć Neuronowa                    134963
8           Metoda Kubańska                                   128843
9           Fifty-Fifty                                       127722
10          DarwoX (liczby pierwsze)                          126710
11          Trynidad_Tobago                                   110466
12          Foch Forever                                      1

In [15]:
import pandas as pd

# Zamiana słownika results na DataFrame
df = pd.DataFrame(
    sorted(results.items(), key=lambda x: x[1], reverse=True),
    columns=["Strategia", "Punkty"]
)

# Dodanie numeru miejsca
df.insert(0, "Miejsce", range(1, len(df) + 1))

# Zapis do pliku Excel
df.to_excel("wyniki_turnieju_3_1.xlsx", index=False)

print("Plik 'wyniki_turnieju_3_1.xlsx' został zapisany.")

Plik 'wyniki_turnieju_3_1.xlsx' został zapisany.


### Wersja przestrzenna

In [8]:
import random
import os
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

In [16]:
# =============================
# PARAMETRY SYMULACJI
# =============================

N = 51                 # rozmiar planszy NxN
N_ITERATIONS = 100     # liczba iteracji gry
DELTA = 0.95            # prawdopodobieństwo kolejnej rundy
OUTPUT_DIR = "frames"   # folder na obrazki

MACIERZ_WYPLAT = {
    ('L', 'L'): (-3, -3),
    ('L', 'Z'): (-10, 0),
    ('Z', 'L'): (0, -10),
    ('Z', 'Z'): (-5, -5)
}

In [18]:
def play_iterated_pd(p1, p2, payoff_matrix, delta):
    h1, h2 = [], []
    s1, s2 = 0, 0
    rounds = 0

    p1.reset()
    p2.reset()

    while True:
        a1 = p1.ruch(h1, h2)
        a2 = p2.ruch(h2, h1)

        r1, r2 = payoff_matrix[(a1, a2)]
        s1 += r1
        s2 += r2

        h1.append(a1)
        h2.append(a2)
        rounds += 1

        if random.random() > delta:
            break

    return s1 / rounds, s2 / rounds

def initialize_grid(n, strategy_classes):
    strategies = []
    total = n * n
    k = len(strategy_classes)

    for cls in strategy_classes:
        strategies.extend([cls(cls.__name__)] * (total // k))

    random.shuffle(strategies)

    grid = []
    idx = 0
    for i in range(n):
        row = []
        for j in range(n):
            row.append(strategies[idx])
            idx += 1
        grid.append(row)

    return grid

def initialize_grid_stripes_vertical(n, strategy_classes):
    k = len(strategy_classes)
    stripe_width = n // k

    grid = [[None for _ in range(n)] for _ in range(n)]

    col = 0
    for cls in strategy_classes:
        for _ in range(stripe_width):
            if col >= n:
                break
            for row in range(n):
                grid[row][col] = cls(cls.__name__)
            col += 1

    # jeśli n nie dzieli się idealnie przez k – resztę wypełnij losowo
    remaining = [
        cls(cls.__name__) for cls in strategy_classes
        for _ in range(n)
    ]
    random.shuffle(remaining)

    for i in range(n):
        for j in range(n):
            if grid[i][j] is None:
                grid[i][j] = remaining.pop()

    return grid

def initialize_checkerboard(n, strategy_classes):
    k = len(strategy_classes)
    grid = []

    for i in range(n):
        row = []
        for j in range(n):
            idx = (i + j) % k
            row.append(strategy_classes[idx](strategy_classes[idx].__name__))
        grid.append(row)

    return grid

def initialize_islands(n, strategy_classes, island_size=5):
    import random

    base = strategy_classes[0]
    grid = [[base(base.__name__) for _ in range(n)] for _ in range(n)]

    for cls in strategy_classes[1:]:
        x = random.randint(0, n - island_size)
        y = random.randint(0, n - island_size)

        for i in range(island_size):
            for j in range(island_size):
                grid[x+i][y+j] = cls(cls.__name__)

    return grid

def initialize_single_defector_center(n, CoopClass, DefectClass):
    """
    Cała plansza = współpraca,
    tylko środkowy gracz = zdrada
    """
    assert n % 2 == 1, "n musi być nieparzyste, żeby istniał dokładny środek"

    grid = [
        [CoopClass(CoopClass.__name__) for _ in range(n)]
        for _ in range(n)
    ]

    center = n // 2
    grid[center][center] = DefectClass(DefectClass.__name__)

    return grid

def initialize_rings(n, strategy_classes):
    k = len(strategy_classes)
    center = n // 2
    grid = []

    for i in range(n):
        row = []
        for j in range(n):
            dist = abs(i - center) + abs(j - center)
            idx = (dist // 3) % k
            row.append(strategy_classes[idx](strategy_classes[idx].__name__))
        grid.append(row)

    return grid

def get_neighbors(i, j, n):
    for di, dj in [(-1,0),(1,0),(0,-1),(0,1)]:
        ni, nj = i + di, j + dj
        if 0 <= ni < n and 0 <= nj < n:
            yield ni, nj

def initialize_spiral(n, strategy_classes):
    k = len(strategy_classes)
    cx = cy = n // 2
    grid = [[None]*n for _ in range(n)]

    x, y = cx, cy
    dx, dy = 0, -1
    steps = 1
    step_count = 0
    turns = 0

    for s in range(n*n):
        idx = s % k
        grid[x][y] = strategy_classes[idx](strategy_classes[idx].__name__)

        if step_count == steps:
            dx, dy = -dy, dx
            step_count = 0
            turns += 1
            if turns % 2 == 0:
                steps += 1

        x += dx
        y += dy
        step_count += 1

        if not (0 <= x < n and 0 <= y < n):
            break

    return grid

def play_iteration(grid):
    n = len(grid)
    results = [[[] for _ in range(n)] for _ in range(n)]

    played = set()

    for i in range(n):
        for j in range(n):
            for ni, nj in get_neighbors(i, j, n):
                pair = tuple(sorted([(i,j),(ni,nj)]))
                if pair in played:
                    continue

                played.add(pair)

                p1 = grid[i][j]
                p2 = grid[ni][nj]

                r1, r2 = play_iterated_pd(p1, p2, MACIERZ_WYPLAT, DELTA)

                results[i][j].append(r1)
                results[ni][nj].append(r2)

    avg_scores = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            avg_scores[i][j] = sum(results[i][j]) / len(results[i][j])

    return avg_scores

def update_strategies(grid, avg_scores):
    n = len(grid)
    new_grid = [[grid[i][j] for j in range(n)] for i in range(n)]

    for i in range(n):
        for j in range(n):
            best_score = avg_scores[i][j]
            best_strategy = grid[i][j]

            for ni, nj in get_neighbors(i, j, n):
                if avg_scores[ni][nj] > best_score:
                    best_score = avg_scores[ni][nj]
                    best_strategy = grid[ni][nj]

            new_grid[i][j] = type(best_strategy)(best_strategy.name)

    return new_grid

def initialize_biased_random(n, strategy_classes, weights):
    import random
    assert len(weights) == len(strategy_classes)

    grid = []
    for i in range(n):
        row = []
        for j in range(n):
            idx = random.choices(range(len(strategy_classes)), weights)[0]
            row.append(strategy_classes[idx](strategy_classes[idx].__name__))
        grid.append(row)

    return grid

STRATEGIES = [
    "ZawszeLojalnosc",
    "ZawszeZdrada",
    "WetZaWet",
    "FochForever",
    "FiftyFifty",
    "TrynidadTobago",
    "MetodaKubanska",
    "PrawieSiecNeuronowa",
    "DarwoX",
    "PrzebaczajacyDoCzasu",
    "Reklama",
    "NieLubiPrzegrywać",
    "KubaSubcio",
    "PNANG",
    "OdsiewFrajerow",
    "Jedikarix",
    "Krukindzek",
    "OrzechowaSzansa",
    "PruskiKwadrat",
    "Dwulicowiec",
    "ShiZ",
    "ILike3s",
    "WiaraWLudzkosc"
]

STRATEGY_TO_INT = {name: i for i, name in enumerate(STRATEGIES)}

INT_TO_COLOR = [
    "#1f77b4",  # Zawsze Lojalność (niebieski)
    "#d62728",  # Zawsze Zdrada (czerwony)
    "#2ca02c",  # Wet Za Wet (zielony)
    "#8c564b",  # Foch Forever (brąz)
    "#ff7f0e",  # Fifty-Fifty (pomarańczowy)
    "#9467bd",  # Trynidad_Tobago (fiolet)
    "#17becf",  # Metoda Kubańska (turkus)
    "#bcbd22",  # Sieć Neuronowa (oliwkowy)
    "#7f7f7f",  # DarwoX (szary)
    "#aec7e8",  # Przebaczający (jasny niebieski)
    "#ff9896",  # Reklama (jasna czerwień)
    "#c49c94",  # Nie Lubi Przegrywać (beż)
    "#98df8a",  # Kuba Subcio (jasna zieleń)
    "#dbdb8d",  # PNANG (piaskowy)
    "#e377c2",  # Odsiew Frajerów (róż)
    "#f7b6d2",  # Jedikarix (jasny róż)
    "#9edae5",  # Krukindzek (jasny turkus)
    "#c7c7c7",  # Orzechowa Szansa (jasny szary)
    "#393b79",  # Pruski Kwadrat (ciemny granat)
    "#843c39",  # Dwulicowiec (ciemna czerwień)
    "#637939",  # ShiZ (ciemna oliwka)
    "#8c6d31",  # I Like 3's (musztardowy)
    "#3182bd"   # Wiara w Ludzkość (czysty niebieski)
]

def add_fixed_legend():
    handles = [
        mpatches.Patch(color=INT_TO_COLOR[i], label=name)
        for name, i in STRATEGY_TO_INT.items()
    ]
    plt.legend(handles=handles, loc="upper right")
    
def save_grid_image(grid, iteration):
    n = len(grid)

    img = [
        [STRATEGY_TO_INT[grid[i][j].name] for j in range(n)]
        for i in range(n)
    ]

    cmap = ListedColormap(INT_TO_COLOR)
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # --- CZARNE TŁO ---
    fig.patch.set_facecolor("black")
    ax.set_facecolor("black")
    
    plt.imshow(
        img,
        cmap=cmap,
        vmin=0,
        vmax=len(INT_TO_COLOR) - 1
    )
    plt.title(f"Iteracja {iteration+1}",color="white",fontsize=16,pad=14)
    plt.axis("off")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    plt.savefig(f"{OUTPUT_DIR}/iter_{iteration:04d}.png")
    plt.close()

In [22]:
strategy_classes = [
    ZawszeLojalnosc,
    ZawszeZdrada,
    WetZaWet#,
    #FochForever,
    #FiftyFifty,
    #TrynidadTobago,
    #MetodaKubanska,
    #PrawieSiecNeuronowa,
    #DarwoX,
    #PrzebaczajacyDoCzasu,
    #Reklama,
    #NieLubiPrzegrywać,
    #KubaSubcio,
    #PNANG,
    #OdsiewFrajerow,
    #Jedikarix,
    #Krukindzek,
    #OrzechowaSzansa,
    #PruskiKwadrat,
    #Dwulicowiec,
    #ShiZ,
    #ILike3s,
    #WiaraWLudzkosc
]

grid = initialize_grid_stripes_vertical(N, strategy_classes)

for t in range(N_ITERATIONS):
    save_grid_image(grid, t)
    scores = play_iteration(grid)
    grid = update_strategies(grid, scores)

print("Symulacja zakończona.")

Symulacja zakończona.


### Legenda

In [50]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

INT_TO_COLOR = [
    "#1f77b4",  # Zawsze Lojalność (niebieski)
    "#d62728",  # Zawsze Zdrada (czerwony)
    "#2ca02c",  # Wet Za Wet (zielony)
    "#8c564b",  # Foch Forever (brąz)
    "#ff7f0e",  # Fifty-Fifty (pomarańczowy)
    "#9467bd",  # Trynidad_Tobago (fiolet)
    "#17becf",  # Metoda Kubańska (turkus)
    "#bcbd22",  # Sieć Neuronowa (oliwkowy)
    "#7f7f7f",  # DarwoX (szary)
    "#aec7e8",  # Przebaczający (jasny niebieski)
    "#ff9896",  # Reklama (jasna czerwień)
    "#c49c94",  # Nie Lubi Przegrywać (beż)
    "#98df8a",  # Kuba Subcio (jasna zieleń)
    "#dbdb8d",  # PNANG (piaskowy)
    "#e377c2",  # Odsiew Frajerów (róż)
    "#f7b6d2",  # Jedikarix (jasny róż)
    "#9edae5",  # Krukindzek (jasny turkus)
    "#c7c7c7",  # Orzechowa Szansa (jasny szary)
    "#393b79",  # Pruski Kwadrat (ciemny granat)
    "#843c39",  # Dwulicowiec (ciemna czerwień)
    "#637939",  # ShiZ (ciemna oliwka)
    "#8c6d31",  # I Like 3's (musztardowy)
    "#3182bd"   # Wiara w Ludzkość (czysty niebieski)
]

STRATEGY_NAMES = [
    "Zawsze Lojalność",
    "Zawsze Zdrada",
    "Wet Za Wet",
    "Foch Forever",
    "Fifty-Fifty",
    "Trynidad Tobago",
    "Metoda Kubańska",
    "Tutaj Miała Być Sieć Neuronowa",
    "DarwoX",
    "Przebaczający Do Czasu",
    "Sprawdź kanał autora (Paradoxalnie)",
    "Nie Lubi Przegrywać",
    "Kuba Subcio",
    "Pozytywnie Nastawieni Ale Nieufni Gracze",
    "Odsiew Frajerów",
    "Jedikarix",
    "Krukindzek",
    "Orzechowa Szansa",
    "Pruski Kwadrat",
    "Dwulicowiec",
    "ShiZ",
    "I Like 3's",
    "Wiara w Ludzkość"
]

def build_legend_handles():
    return [
        mpatches.Patch(color=INT_TO_COLOR[i], label=STRATEGY_NAMES[i])
        for i in range(len(STRATEGY_NAMES))
    ]

def save_legend_only(filename="legend.png", columns=1, black_bg=True):
    patches = [
        mpatches.Patch(color=INT_TO_COLOR[i], label=STRATEGY_NAMES[i])
        for i in range(len(STRATEGY_NAMES))
    ]

    fig, ax = plt.subplots(figsize=(6, len(patches) * 0.35))
    ax.axis("off")

    legend = ax.legend(
        handles=patches,
        loc="center",
        ncol=columns,
        frameon=False
    )

    if black_bg:
        fig.patch.set_facecolor("black")
        for text in legend.get_texts():
            text.set_color("white")

    plt.savefig(filename, bbox_inches="tight", dpi=200)
    plt.close()

In [56]:
save_legend_only("legenda_strategii_3.png", columns=1)

### Renderowanie

In [12]:
import os
import imageio.v2 as imageio

def images_to_mp4(
    folder,
    output_file="output.mp4",
    fps=10,
    prefix="iter_"
):

    files = sorted(
        f for f in os.listdir(folder)
        if f.startswith(prefix) and f.lower().endswith((".png", ".jpg", ".jpeg"))
    )

    if not files:
        raise ValueError("Nie znaleziono żadnych plików pasujących do wzorca.")

    output_path = os.path.join(folder, output_file)

    with imageio.get_writer(output_path, fps=fps, codec="libx264", format="ffmpeg") as writer:
        for filename in files:
            image_path = os.path.join(folder, filename)
            image = imageio.imread(image_path)
            writer.append_data(image)

    print(f"Film zapisany jako: {output_path}")
    print(f"Liczba klatek: {len(files)}, FPS: {fps}")

In [48]:
images_to_mp4(
    folder="spatial_example4",
    output_file="spatial_example4.mp4",
    fps=5
)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Film zapisany jako: symulacja przestrzenna 1.3.5\symulacja przestrzenna 1.3.4.mp4
Liczba klatek: 300, FPS: 5
